# Mod 2 Review

In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from statsmodels.formula.api import ols

pd.set_option('display.max_columns', 100)

### Check Your Data … Quickly
The first thing you want to do when you get a new dataset, is to quickly to verify the contents with the .head() method.

In [87]:
df = pd.read_csv('movie_metadata.csv')
print(df.shape)
df.head()

df.columns

(5043, 28)


Index(['color', 'director_name', 'num_critic_for_reviews', 'duration',
       'director_facebook_likes', 'actor_3_facebook_likes', 'actor_2_name',
       'actor_1_facebook_likes', 'gross', 'genres', 'actor_1_name',
       'movie_title', 'num_voted_users', 'cast_total_facebook_likes',
       'actor_3_name', 'facenumber_in_poster', 'plot_keywords',
       'movie_imdb_link', 'num_user_for_reviews', 'language', 'country',
       'content_rating', 'budget', 'title_year', 'actor_2_facebook_likes',
       'imdb_score', 'aspect_ratio', 'movie_facebook_likes'],
      dtype='object')

## Question 1
A Hollywood executive whats to know how much a R-Rated movie released after 2000 will earn. The data above is a sample of some of the movies with that ranting during that time frame, as well as other movies. How would you go about answering her question? Talk through it theoretically and then do it in code.

#### First clean the data to find the R rated movies with revenue, find the profits by adding a colum that is the gross - budget. Make sure nan values are not counted in n and in those profits or budgets with nan values when subtracting them. Using a 95% level of confidence or significance of 0.025 to find the two tail z test with a total alpha of 0.05 and divide by 2 to find the min and max confidence intervals. ####


In [191]:
from scipy import stats

mean  = df[(df['title_year']>2000) &(df['content_rating']=='R')]['gross'].mean()

stddev  = df[(df['title_year']>2000) &(df['content_rating']=='R')]['gross'].std()

In [192]:
n = len(df[(df['title_year']>2000) &(df['content_rating']=='R')])


In [193]:
_df = df[(df['title_year']>2000) &(df['content_rating']=='R')].copy()

In [195]:
minus = mean - 1.96*(stddev/n**.5)
plus = mean - 1.96*(stddev/n**.5)
plus, minus


#-48965307.24992396

(25593896.732384615, 25593896.732384615)

In [93]:
stats.t.interval(alpha = 0.05, 
                df = len(_df)-1,
                 loc = mean,
                 scale = stddev)


(25197269.346510723, 30100427.52931709)

In [94]:
#Confidence interval at 95%

In [95]:
#if there is a nan in 

In [197]:
import numpy as np

_df['profit'] = _df['gross'] - _df['budget']

In [198]:
_df.profit = np.where((_df.budget.isnull()) | (_df.gross.isnull()), np.nan, _df.profit)

In [199]:
mean = _df['profit'].mean()
stddev = _df['profit'].std()
mean

-14114037.355535554

In [204]:
stats.norm.interval(alpha = 0.05,
                   loc = mean, 
                   scale = stddev)

(-39028279.18019109, 10800204.469119987)

In [201]:
#104263004.12575178
#-48965307.24992396

In [202]:
from scipy import stats as scs

mean_ = _df.profit.mean()
std_ = _df.profit.std()
alpha = 0.05
n = len(_df.dropna(subset = ['profit']))

zscore = scs.norm.ppf(1-(alpha/2))

max_ = mean_+ zscore * (std_/np.sqrt(n))
min_ = mean_ - zscore * (std_/np.sqrt(n))

In [203]:
n

1111

In [182]:
max_, min_

(9248728.208643755, -37476802.91971486)

In [183]:
mean_

-14114037.355535554

In [189]:
minus = mean - 1.96*(stddev/n**.5)


plus = mean + 1.96*(stddev/n**.5)
plus, minus


(9249157.512832018, -37477232.22390313)

## Question 2

Your ability to answer the first question has the executive excited and now she has many other questions about the types of movies being made and the differences in those movies budgets and gross amounts.  Read through the questions below and determine what type of statistical test you should use for each question and write down the null and alternative hypothesis for those tests.    

- Is there a realtionship between the number of facebook likes for a cast and the box office gross of the movie?
- Do foreign films perform differently at the box office than non-foreign films?
- Of all movies created are 40% rated R?
- Is there a relationship between the language of a film and the content rating (G, PG, PG-13, R)of that film?
- Is there a relationship between the content rating of a film and its budget? 

#### Is there a realtionship between the number of facebook likes for a cast and the box office gross of the movie? ####

Number of Likes: Continuous
Box office Gross: Continuous

Relationship between two continuous values, test: 

Pearson Correlation CoEfficient
or 
Linear Regression

Is there a relationship between the content rating of a film and its budget?


Test: 
    H0 No correlation
    Ha There is correlation




In [120]:
import seaborn as sns


In [123]:
foreign.sem(axis = 0)

167274881.23395962

#### Do foreign films perform differently at the box office than non-foreign films? ####

#foreign film box office revenue mean: Continuous
#non-foreign box office revenue mean: Continuous

#Compare two means of the revenue of two quantitative groups
#two sample t-test of means 


#Test: 
    #H0 mean1-mean2 = 0
    #Ha mean1-mean2 != 0

In [144]:
foreign = _df[_df.language != 'English']['profit']
domestic = _df[_df.language == 'English']['profit']
alpha = 0.05



def independent_ttest(df1, df2, alpha):

    #calculate means
    mean1, mean2 = df1.mean(), df2.mean()

    # calculate standard errors
    se1, se2 = df1.sem(axis = 0), df2.sem(axis = 0)

    # standard error on the difference between the samples
    sed = np.sqrt(se1**2 + se2**2)
    # calculate the t statistic
    t_stat = ((df1.mean()) - (df2.mean()) )/ sed

    # degrees of freedom (n1+n2)-2
    dof = len(df1) + len(df2) - 2

    # calculate the critical value
    cv = stats.t.ppf(1.0 - alpha, dof)

    # calculate the p-value
    p = (1.0 - stats.t.cdf(abs(t_stat), dof)) * 2.0
    
    # return the good stuff
    return 't-stat: ',t_stat, 'degrees of freedom: ',dof, 'critical-value', cv, 'pvalue', p

cv = 1.6459521808844497
t_stat = -1.7095288156536121


        
        
independent_ttest(foreign, domestic, alpha)

        
 

('t-stat: ',
 -1.7095288156536121,
 'degrees of freedom: ',
 1388,
 'critical-value',
 1.6459521808844497,
 'pvalue',
 0.08757655539551701)

In [172]:
def reject_fail_to_reject(t_stat, cv, alpha):
    if abs(t_stat) <= cv:
        print ('Fail to reject the null hypothesis that the means are equal at a significance level of ', alpha, '.')
    else:
        print ('Reject the null hypothesis that the means are equal at a significance level of ', alpha, '.')

In [173]:


foreign = _df[_df.language != 'English']['profit']
domestic = _df[_df.language == 'English']['profit']


In [174]:
foreign.dropna(inplace = True)
domestic.dropna(inplace = True)

In [175]:

scs.ttest_ind(foreign, domestic) 
# two sample independent t-test

Ttest_indResult(statistic=-6.233110236021216, pvalue=6.486026348275082e-10)

In [176]:
t_stat = -6.233110236021216
p-value = 6.486026348275082e-10
alpha = 0.05

reject_fail_to_reject(t_stat, cv, alpha)

SyntaxError: can't assign to operator (<ipython-input-176-f795f1e4b911>, line 2)

#### Of all movies created are 40% rated R? ####

proportion of movies
specifically you want to do a two tail difference of proportions
proportions always z test, there is no use of std


Test:

    H0 P = .40
    Ha P != .40
    prove it is not above or below .4




#### Is there a relationship between the language of a film and the content rating (G, PG, PG-13, R)of that film? ####

language: catergorical
content rating: catergorical

Two catergories to compare
Chi Square, Frequency

Test: 
    H0 no relationship
    Ha relationship

In [154]:


language_rating = _df['language','rating]'
scs.chi2_contingency(language_rating) 



SyntaxError: invalid syntax (<ipython-input-154-a0248981da82>, line 2)

#### Is there a relationship between the content rating of a film and its budget?####

Content Rating catergorical
Budget is continuous 

H0 mean budget per catergory is =
Ha at least one mean budget of a rating !=


ANOVA 

## Question 3

Now that you have answered all of those questions, the excutive wants you to create a model that predicts the money a movie will make if it is released next year in the US. She wants to use this to evaluate different scripts and then decide which one has the largest revenue potential. Below is a list of potential features you could use in the model. Discuss the steps you would need to take in order to prepare the data for the model. Would you use all of these features in the model? Identify which features you might drop and why, and discuss the process you would use to determine this.  

*Remember you want to be able to use this model to predict the box office gross of a film **before** anyone has seen it.*

- budget: The amount of money spent to make the movie
- title_year: The year the movie first came out in the box office
- years_old: How long has it been since the movie was released
- genre: Each movie is assigned one genre category like action, horror, comedy.
- avg_user_rating: This rating is taken from Rotten tomatoes, and is the average rating given to the movie by the audience. 
- actor_1_facebook_likes: The number of likes that the most popular actor in the movie has
- total_cast_facebook_likes: The sum of likes for the three most popular actors in the movie
- language: the orginal spoken language of the film


In [ ]:
mlr_model = ols(fomrula = 'gross-cast_total')

In [ ]:
title year or years old drop they would produce multicollinearity
actor likes and cast likes facebook take one way becasue they correlate too much also could produce multicollinarity 
(you could keep one of each)

rating is unknown because it is not out yet 

## Question 4

Below is the summary output of a fitted OLS model.  Look at the summary of this OLS model and identify any key takeaways from it.  How ‘good’ is this model? Which features help to explain the variance in the target variable? Which do not? 


<img src="ols_summary.png" style="withd:300px;">

In [ ]:
skew higher than one and kurtosis higher than three show it is highly skewed
r2 shows there's not much correlation shown
adjusted isn't much different
pvalues, some less than 0.05 
intersections 

skewness higher than 1 skewed 
higher than 1 positive skew
less than -1 negative



R2 low, very little variance is explained by your targets using the features that you have
if p value is over 0.05 (level of significance can change, but in this case 0.05)
p value is dependent on the over features

if you drop a feature, it may change the p value and be less or more influential of your target 



## Question 5

**Bayes Theorem**

An advertising executive is studying television viewing habits of married men and women during prime time hours. Based on the past viewing records he has determined that during prime time wives are watching television 60% of the time. It has also been determined that when the wife is watching television, 40% of the time the husband is also watching. When the wife is not watching the television, 30% of the time the husband is watching the television. Find the probability that if the husband is watching the television, the wife is also watching the television.

In [78]:
#the wife is also watching the television if the husband is watching the television, 

#P(A) = P(wife) = Wife watching
#P(B) = P(husband) = Husband watching

p_wife = .60
p_wife_not = .40

p_husband_given_wife = .40
p_husband_given_not_wife = .30

p_husband = ((p_husband_given_wife) * p_wife) + ((p_husband_given_not_wife)* p_wife_not)
p_husband

# P(B) = P(B|A)* P(A) + P(B|A') * P(A')



#if denotes B
#suppose denotes B





0.36

In [77]:
# P(A|B) = P(B|A) * P(A) / P(B)

wife_given_husband = (p_husband_given_wife * p_wife)/p_husband


# P(A|B)
print ('P(A|B) is : ', wife_given_husband)

P(A|B) is :  0.6666666666666666


## Question 6

Explain what a type 1 error is and how does it relate to the significance level when doing a statistical test. 

A type 1 error is a false positive. (Meme example: A man beingn told he is pregnant) 
We Reject H0 when we should have failed to reject H0.  

A type 2 error is a false negative. (Meme example: a woman is told she is not pregnant when clearly she is)
When we Fail to Reject H0 even though we should have rejected H0. 
